In [8]:
import production as p
import os
import inspect
import sys
import pandas as pd

# access parent directory from notebooks directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir)
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

### Step1: Read the LLM-ready data

In [36]:
df_user_data,neutral_sentences = p.read_input_data()

In [37]:
df_user_data.head(3)

,username,messageID,original
49,U0,29,"I have a younger sister, 22 now, and a mother ..."
47,U0,20,I am gonna sound very young when I tell you ab...
46,U0,2,So there is one that we have in common and tha...


In [38]:
neutral_sentences

,idSentence,sentences
0,0,I’m all about that food. I usually kick off th...
1,1,Just getting my vitamins in at the school cant...
2,2,De Pizzabakkers sell this vegan pizza with che...
3,3,I entered the world of vegan foods lately. Nex...
4,4,This vegan fried chicken from KFC is on the sp...
5,5,Just having this vegan hotdog from the school ...
6,6,"Just had my first vegan cake at ""groene bakker..."
7,7,This vegan chocolate is on point. Its with oat...
8,8,I'm all about fast food. I'm having some fries...
9,9,"I went to the ""Groene Burger"" fast food restau..."


### Step 2 LLM TST

In [101]:
# This calls Mistral API
df_llm_tst = p.llm_tst(df_user_data,neutral_sentences[0:2])
df_llm_tst_final = p.postprocess_llm_tst(df_llm_tst)
df_llm_tst_final.head(3)

In [25]:
# This gets the LLM TST data already generated
df_llm_tst_final = p.read_and_postprocess_llm_tst()

from datetime import datetime
smallest_timestamp = datetime.strptime(str(df_llm_tst_final['timestamp'].min()), "%Y%m%d%H%M%S").strftime("%B %d %H:%M:%S")

largest_timestamp = datetime.strptime(str(df_llm_tst_final['timestamp'].max()), "%Y%m%d%H%M%S").strftime("%B %d %H:%M:%S")
print(smallest_timestamp,largest_timestamp)

May 20 12:02:53 May 24 19:05:49


In [26]:
df_llm_tst_final.columns

Index(['username', 'id_neutral_sentence', 'neutral_sentence', 'tst_id',
       'tst_sentence', 'explanation', 'llm_tst', 'query', 'model',
       'prompt_tokens', 'completion_tokens', 'object', 'promptID',
       'timestamp'],
      dtype='object')

### Step 3 LLM Evaluate

In [167]:
df_eval = p.llm_evl(df_llm_tst_final)

##### This gets the LLM EVL data already generated

In [9]:
df_eval = pd.read_csv('f8_llm_evaluation_data/eval_20240520173356.csv')

In [10]:
df_eval.head(3)

,tst_id,tst_sentence,eval_id,llm_eval,query,model,prompt_tokens,completion_tokens,object,eval_promptID,eval_timestamp
0,U020240520120253,"I'm totally into food, you know? I start my we...",U020240520173219,score_formality=1\nexplanation=The text contai...,You are a lingustics expert.Your goal is to as...,mistral-small,238,66,chat.completion,0,20240520173219
1,U020240520120253,"I'm totally into food, you know? I start my we...",U020240520173221,score_descriptiveness=2\nexplanation=The text ...,You are a lingustics expert.Your goal is to as...,mistral-small,267,128,chat.completion,1,20240520173221
2,U020240520120253,"I'm totally into food, you know? I start my we...",U020240520173221,score\_emotionality=3\nexplanation=The text is...,You are a lingustics expert.Your goal is to as...,mistral-small,245,69,chat.completion,2,20240520173221


In [11]:
df_eval_final = p.postprocess_llm_evl(df_eval)

Exception at index: 16  
 with value: Score: 2_somewhat concise/descriptive

Explanation: The text provides some descriptive elements, such as the mention of specific food items (quinoa, chickpeas, soy chicken) and the positive attitude towards the upcoming math exam. However, it is still concise and lacks detailed descriptions that could create a vivid image or evoke stronger emotions.
